In [4]:
%pip install qianfan==0.4.10.1 python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
# load from .env QIANFAN_AK and QIANFAN_SK
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import sys
sys.path.append("..")

In [3]:
from eval.dataset import RAGDataset

ds = RAGDataset.from_file("../data/airbench_qa_healthcare_zh.json")

In [8]:
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint
from langchain_core.caches import InMemoryCache
from langchain.globals import set_llm_cache

# add llm cache to avoid duplicate calls
set_llm_cache(InMemoryCache())

llm = QianfanLLMEndpoint(
    model="ERNIE-Speed-Pro-128K",
    temperature=0.001,
    init_kwargs={
        # https://cloud.baidu.com/doc/WENXINWORKSHOP/s/Llroa77io
        "request_per_minute": 9000, "token_per_minute": 700000}
)

In [9]:
from langchain_core.language_models import LLM
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm
import json

QUESTIONS_GENERATE_PROMPT_TPL = """\
上下文信息如下。
---------------------
{context_str}
---------------------

你是一位教师/教授。您的任务是为即将到来的测验/考试设置 {num_questions_per_chunk} 个问答对。

- 这些问答对应在整个文档中均匀分布。
- 将问题和答案限制在所提供的上下文信息中。
- 每个问题只能包含一个提问点。
- 仅根据上下文信息，而非先验知识。
- 请以JSON格式输出问答对，"问题" 和 "答案" 为字段，必须严格遵守 JSON 格式，格式如下：

```json
[
  {{"问题": "问题1", "答案": "答案1"}},
  {{"问题": "问题2", "答案": "答案2"}},
  ...
]
```
"""

chain = PromptTemplate.from_template(QUESTIONS_GENERATE_PROMPT_TPL) | llm | JsonOutputParser()

In [11]:
chain.invoke(dict(
    context_str=ds.corpus[list(ds.corpus.keys())[0]],
    num_questions_per_chunk=10
))

[{'问题': '激光去痘印的原理是什么？',
  '答案': '激光去痘印的原理是应用激光仪产生的高强度的光束，激光类型不同产生光线颜色不同。这种去痘印方法采用的激光是治疗的关键。通过激光能量令染料颗粒崩解汽化，封闭血管，再由身体吸收染料颗粒将其排除体外，色素随之消退。'},
 {'问题': '激光去痘印的价格范围是多少？',
  '答案': '医院祛痘印的价格一般是不贵的，大概的手术的价格3000元-6000元，手术费用的多少与你所选择的医院的收费标准及手术面积的严重程度有关。'},
 {'问题': '激光去痘印需要多长时间才能看到效果？', '答案': '激光去痘印一般1~3个月看到效果，需要多次激光治疗。'},
 {'问题': '激光治疗痤疮疤痕能减少哪些皮肤缺陷？', '答案': '激光治疗痤疮疤痕能减少皱纹等皮肤缺陷。'},
 {'问题': '皮秒激光祛痘印有什么特点？',
  '答案': '皮秒激光祛痘印是一种纯物理的治疗方式，不对皮肤没有任何刺激，没有光敏现象，激光对正常的周围皮肤是不产生作用，治疗过程轻松，无任何痛苦。'},
 {'问题': '皮秒激光祛痘印后皮肤颜色会如何变化？',
  '答案': '治疗后皮肤的颜色会变得越来越接近正常皮肤颜色，个别可能会比正常的皮肤颜色深一些。再经过几个月的恢复后，会和原来的皮肤颜色差不多。'},
 {'问题': '激光治疗痤疮疤痕后需要特殊护理吗？', '答案': '激光治疗痤疮疤痕后不需要特殊护理，但应避免强烈日晒和紫外线照射。'},
 {'问题': '激光去痘印能够直接穿透到哪里？', '答案': '激光去痘印能够直接穿透的部位形成矩阵状的小白点。'},
 {'问题': '小白点周围的皮肤在多长时间内会愈合？', '答案': '小白点周围的皮肤立即启动横向修复机制，在8小时以内表皮就会完全愈合。'},
 {'问题': '激光去痘印手术需要多长时间恢复？', '答案': '几天左右，小白点处的痂皮脱落，皮肤恢复正常外观。'}]

In [12]:
# Batch invoke and fault tolerance
# 测试时可以改成5 
CORPUS_SAMPLE_SIZE = 1000
import random
random.seed(42)
# 从 corpus 中随机选择 1000 个文档，每个文档生成 10 个问答对
corpus_sample = random.sample(list(ds.corpus.items()), CORPUS_SAMPLE_SIZE)

from concurrent.futures import ThreadPoolExecutor

def generate_questions(args):
    corpus_id, corpus_content = args['corpus']
    num_questions_per_chunk = args['num_questions_per_chunk']
    try:
        ret = chain.invoke(dict(
            context_str=corpus_content,
            num_questions_per_chunk=num_questions_per_chunk
        ))
    except Exception as e:
        print(f"{corpus_id} API调用失败: {e}")
        return dict(corpus_id=corpus_id, questions=[], answer=[])
    questions = []
    answers = []
    for qa in ret:
        # [{'问题': '什么类型的激光适合治疗不同的痘印？', '答案': '不同类型的激光产生不同颜色光线，适合治疗不同类型的痘印。选择适合的激光类型需要根据个人肤质和痘印情况由专业医生进行判断。'}]]
        if qa and qa.get('问题') and qa.get('答案'):
            questions.append(qa['问题'])
            answers.append(qa['答案'])
    return dict(corpus_id=corpus_id, questions=questions, answer=answers)
    

# 10 并发调用
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [
        executor.submit(generate_questions, dict(corpus=corpus, num_questions_per_chunk=10))
        for corpus in corpus_sample
    ]
    results = [future.result() for future in tqdm(futures, total=len(corpus_sample))]

doc-210325 API调用失败: Invalid json output: ```json
[
  {"问题": "婴儿智力低下的早期表现有哪些？", "答案": "婴儿智力低下的早期表现包括：1. 出生后几个月的婴儿非常老实、安静、不哭不闹，整天昏昏欲睡。2. 出生2个月还不会笑。3. 两只眼睛对周围的人和物视而不见，无反应、无兴趣。4. 对周围的声音缺乏反应。5. 正常婴儿3～5个月时，常常会在眼前玩弄自己的双手，反复端详，6个月后便逐渐消失，而智力低下婴儿6个月以后还继续存在这种动作。6. 正常婴儿6个月以后喂他固体食物时，就会反射性地产生咀嚼动作，而智力低下婴儿很晚才会咀嚼。7. 二三岁时，还常常把玩具、积木放在口里。8. 三四岁后还常常流口水。9. 清醒时也出现磨牙动作。10. 笑声发直，音调缺乏变化。"}
  {"问题": "婴儿智力低下时，会表现出怎样的睡眠习惯？", "答案": "婴儿智力低下时，可能会表现为非常老实、安静、不哭不闹，整天昏昏欲睡。"}
  {"问题": "婴儿出生多久后不会笑可能是智力低下的表现？", "答案": "婴儿出生2个月还不会笑可能是智力低下的表现。"}
  {"问题": "智力低下的婴儿对周围的声音会有什么反应？", "答案": "智力低下的婴儿对周围的声音缺乏反应。"}
  {"问题": "正常婴儿在多大时会在眼前玩弄自己的双手并消失？", "答案": "正常婴儿3～5个月时，常常会在眼前玩弄自己的双手，反复端详，6个月后便逐渐消失。"}
  {"问题": "智力低下的婴儿在多大时还会玩弄自己的双手？", "答案": "智力低下婴儿6个月以后还继续存在这种玩弄自己双手的动作。"}
  {"问题": "正常婴儿在多大时会开始产生咀嚼动作？", "答案": "正常婴儿6个月以后喂他固体食物时，就会反射性地产生咀嚼动作。"}
  {"问题": "智力低下的婴儿在多大才会开始咀嚼？", "答案": "智力低下婴儿很晚才会咀嚼。"}
  {"问题": "智力低下的幼儿在多大时还会把玩具或积木放在口里？", "答案": "二三岁时，智力低下的幼儿还常常把玩具、积木放在口里。"}
  {"问题": "智力低下的幼儿在多大后还会流口水？", "答案": "三四岁后，智力低下的幼儿还常常流口水。"}
]
```
For troub

doc-125140 API调用失败: Invalid json output: ```json
[
  {"问题": "阴道分泌物通常是什么颜色？", "答案": "阴道分泌物通常为白色、淡黄色或绿色。"},
  {"问题": "阴道分泌物可能由什么引起？", "答案": "阴道分泌物可能是由不同时期的月经周期引起，也可能是由感染引起，比如性传播疾病。"},
  {"问题": "医学术语“粘液溢出”指的是什么？", "答案": "“粘液溢出”是指从尿道或阴道流出的黏液分泌物。"},
  {"问题": "“粘液过量溢出”通常指的是什么？", "答案": "“粘液过量溢出”是指黏液分泌物分泌过量，通常特别指的是在淋病中。"},
  {"问题": "淋病中的“粘液过量溢出”是什么症状？", "答案": "在淋病中，“粘液过量溢出”是症状之一，表现为黏液分泌物分泌过量。"},
  {"问题": "阴道分泌物可能有哪些颜色？", "答案": "阴道分泌物可能有的颜色包括白色、淡黄色或绿色。"},
  {"问题": "哪些情况下阴道分泌物可能是正常的？", "答案": "大多数的阴道分泌物都是正常并反映出不同时期的月经周期。"},
  {"问题": "哪些情况下阴道分泌物可能是异常的？", "答案": "一些阴道分泌物可能是因为感染引起，比如性传播疾病，这种情况下阴道分泌物可能是异常的。"},
  {"问题": "什么是“粘液溢出”的医学定义？", "答案": "“粘液溢出”是指从尿道或阴道流出的黏液分泌物。"},
  {"问题": “粘液过量溢出”与什么疾病有关？", "答案": “粘液过量溢出”通常与淋病有关，是淋病的一种症状。"}
]
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE


doc-15217 API调用失败: Invalid json output: ```json
[
  {"问题": "静注时为何需要缓慢进行？", "答案": "静注宜缓慢，以免引起头晕、头胀、胸闷及低血压等。"},
  {"问题": "哪些疾病的患者在发病期应慎用静注？", "答案": "心肌梗死和脑出血在发病期的患者。"},
  {"问题": "静注过程中可能引起哪些症状？", "答案": "头晕、头胀、胸闷及低血压等。"},
  {"问题": "为何心肌梗死和脑出血患者要慎用静注？", "答案": "未提及具体理由，但指出这类患者在发病期应慎用。"},
  {"问题": "静注过程中出现的头晕、头胀等症状与什么有关？, "答案": "与注射速度过快有关。"},
  {"问题": "静注过程中出现的低血压与什么有关？", "答案": "与注射速度过快有关。"},
  {"问题": "哪些情况下需要特别注意静注的速度？, "答案": "在心肌梗死和脑出血发病期，以及一般静注时都需特别注意速度。"},
  {"问题": "静注过程中为何会出现胸闷？", "答案": "可能与注射速度过快导致的不良反应有关。"},
  {"问题": "静注过程中出现的头胀可能与什么有关？", "答案": "可能与注射速度过快导致的不良反应有关。"},
  {"问题": "静注过程中出现的头晕可能与什么有关？", "答案": "可能与注射速度过快导致的不良反应有关。"}
]
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE


100%|██████████| 1000/1000 [09:02<00:00,  1.84it/s]


In [13]:
results[:1]

[{'corpus_id': 'doc-335243',
  'questions': ['脂溢性皮炎主要发生在哪些区域？',
   '脂溢性皮炎的发病机制可能与哪些因素有关？',
   '糠P孢子菌在脂溢性皮炎中起什么作用？',
   '精神因素如何影响脂溢性皮炎的发生和发展？',
   '全身性和难治性脂溢性皮炎可能是哪种感染的皮肤症状？',
   '牛皮癣病变的重要特征是什么？',
   '玫瑰糠疹通常发生在哪些部位？',
   '玫瑰糠疹的母斑有什么特点？',
   '体癣的病变特征是什么？',
   '红斑性天疱疮主要分布在哪些部位？'],
  'answer': ['脂溢性皮炎主要发生在富含皮脂腺的区域，例如头部，躯干等。',
   '脂溢性皮炎的发病机制可能与脂溢性，微生物，神经递质异常，生理和气候因素，营养缺乏和药物的作用有关。',
   '糠P孢子菌，特别是卵形糠P孢子菌，在脂溢性皮炎的发病机理中起重要作用。',
   '精神因素可能对脂溢性皮炎的发生和发展产生一定的影响。',
   '全身性和难治性脂溢性皮炎可能是HIV感染的重要皮肤症状。',
   '牛皮癣病变的重要特征是刮擦鳞片具有薄膜现象和出血现象。',
   '玫瑰糠疹通常发生在颈部，躯干和四肢的近端部分。',
   '玫瑰糠疹的母斑逐渐变大，直径为2-5厘米或更大，有时2-3个母斑可以同时出现。',
   '体癣的病变边缘抬高并变窄，边界清晰，并且有一个圆形损伤，中央愈合延伸到周围区域。',
   '红斑性天疱疮主要分布在面部，颈部，胸部和背部中间。']}]

In [14]:
# save middle results
with open("../data/airbench_qa_healthcare_zh_questions.json", "w") as f:
    json.dump(results, f, ensure_ascii=False)

# load middle results
with open("../data/airbench_qa_healthcare_zh_questions.json", "r") as f:
    results = json.load(f)

In [15]:
# add to dataset train split
if 'train' not in ds.queries_split:
    ds.queries_split['train'] = []
for result in results:
    for i, (question, answer) in enumerate(zip(result['questions'], result['answer'])):
        if len(question) > 5 and len(answer) > 5:
            query_id = f"q-{result['corpus_id']}-{i}"
            ds.queries[query_id] = question
            ds.queries_split['train'].append(query_id)
            ds.relevant_docs[query_id] = [result['corpus_id']]
            ds.reference_answers[query_id] = answer

ds.save("../data/airbench_qa_healthcare_zh_synthesis.json")

In [16]:
len(ds.queries_split['train'])

9810